In [13]:
# Edited to drop clearly uninformative features as is already done in efficient feats notebook
# Apparently there was no need for drop NA or dropping columns where all vals are the same because these did not exist

In [2]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [3]:
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'

data_types = ['Wake','N1', 'N2', 'N3', 'REM']

loaded_data = {}

for data_type in data_types:
    print(data_type)
    X_y_groups = {}
    
    X = pd.read_hdf(load_path + data_type +  'six_second_freq_df.h5', key='df', mode='r')
    y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
    groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
    X, y , groups = X.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

    #Transform the X into TS_Fresh Features___
    # 1. Convert to TS_Fresh format Dataframe 
    ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))
    
    # 2. Extract TS_Fresh Features from the dataframe
    settings = MinimalFCParameters()
    extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)

    # 3. Asign extract_ts_fresh_df to the variable X
    X = extracted_ts_fresh_df.copy()
       

    X_y_groups['X'] = X
    X_y_groups['y'] = y
    X_y_groups['groups'] = groups

    loaded_data[data_type] = X_y_groups

Wake


14-Sep-23 10:47:12 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:01<00:00, 91.58it/s] 


N1


14-Sep-23 10:47:15 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:01<00:00, 72.62it/s]


N2


14-Sep-23 10:47:19 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 57.60it/s]


N3


14-Sep-23 10:47:23 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:01<00:00, 63.06it/s]


REM


14-Sep-23 10:47:26 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:02<00:00, 59.15it/s]


In [15]:
910 / 13 / 7

10.0

In [12]:
## extracted_ts_fresh_df ---> 910 columns
## NA's dropped --> 910 columns
for data_type in data_types:
    X = X_y_groups['X']
    same_value_columns = X.columns[X.nunique() == 1]
    print(data_type) 
    print(same_value_columns)
    print( X.equals(X.dropna(axis = 1) ) )


Wake
Index([], dtype='object')
True
N1
Index([], dtype='object')
True
N2
Index([], dtype='object')
True
N3
Index([], dtype='object')
True
REM
Index([], dtype='object')
True


In [11]:
print(X.equals(X))

True


In [ ]:
 X = X.dropna(axis = 1)

In [7]:
same_value_columns = extracted_ts_fresh_df.columns[extracted_ts_fresh_df.nunique() == 1]
same_value_columns

Index([], dtype='object')

In [6]:
extracted_ts_fresh_df.dropna(axis =1)

,Delta_Central__sum_values,Delta_Central__median,Delta_Central__mean,Delta_Central__length,Delta_Central__standard_deviation,Delta_Central__variance,Delta_Central__root_mean_square,Delta_Central__maximum,Delta_Central__absolute_maximum,Delta_Central__minimum,...,TotalAbsPow_Right Temporal__sum_values,TotalAbsPow_Right Temporal__median,TotalAbsPow_Right Temporal__mean,TotalAbsPow_Right Temporal__length,TotalAbsPow_Right Temporal__standard_deviation,TotalAbsPow_Right Temporal__variance,TotalAbsPow_Right Temporal__root_mean_square,TotalAbsPow_Right Temporal__maximum,TotalAbsPow_Right Temporal__absolute_maximum,TotalAbsPow_Right Temporal__minimum
0,178.415884,0.393769,0.392123,455.0,0.085949,0.007387,0.401432,0.699042,0.699042,0.155450,...,9228.968974,18.621076,20.283448,455.0,7.417813,55.023954,21.597274,86.339395,86.339395,9.804144
1,149.004959,0.405619,0.402716,370.0,0.096114,0.009238,0.414027,0.676764,0.676764,0.125978,...,6116.198975,15.369586,16.530267,370.0,5.423472,29.414050,17.397235,47.523821,47.523821,8.094355
2,447.293735,0.595075,0.577153,775.0,0.117833,0.013885,0.589059,0.812816,0.812816,0.119301,...,9841.634697,11.010547,12.698883,775.0,6.356038,40.399221,14.200735,69.739713,69.739713,4.593473
3,238.784690,0.530446,0.530633,450.0,0.071401,0.005098,0.535415,0.791161,0.791161,0.334278,...,12061.262410,22.259106,26.802805,450.0,14.867840,221.052672,30.650335,138.822564,138.822564,10.076186
4,159.357949,0.525363,0.531193,300.0,0.087329,0.007626,0.538324,0.778927,0.778927,0.342696,...,12325.428963,32.864062,41.084763,300.0,25.975822,674.743303,48.607624,181.248867,181.248867,11.984015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,71.248413,0.623983,0.619551,115.0,0.086390,0.007463,0.625545,0.828271,0.828271,0.369688,...,2615.649540,21.104144,22.744779,115.0,8.226251,67.671205,24.186694,55.614061,55.614061,10.430402
65,139.460655,0.433831,0.435815,320.0,0.137168,0.018815,0.456891,0.806495,0.806495,0.120926,...,4878.818348,13.639040,15.246307,320.0,5.883444,34.614914,16.342117,52.595602,52.595602,6.721379
66,111.876932,0.447400,0.447508,250.0,0.112682,0.012697,0.461476,0.726185,0.726185,0.143943,...,4066.194777,14.336679,16.264779,250.0,8.146573,66.366656,18.190923,71.626224,71.626224,7.924013
67,170.184708,0.495245,0.493289,345.0,0.093314,0.008707,0.502037,0.745341,0.745341,0.145035,...,6727.343118,17.359128,19.499545,345.0,8.830897,77.984741,21.406004,81.915691,81.915691,8.720910


In [5]:
extracted_ts_fresh_df

,Delta_Central__sum_values,Delta_Central__median,Delta_Central__mean,Delta_Central__length,Delta_Central__standard_deviation,Delta_Central__variance,Delta_Central__root_mean_square,Delta_Central__maximum,Delta_Central__absolute_maximum,Delta_Central__minimum,...,TotalAbsPow_Right Temporal__sum_values,TotalAbsPow_Right Temporal__median,TotalAbsPow_Right Temporal__mean,TotalAbsPow_Right Temporal__length,TotalAbsPow_Right Temporal__standard_deviation,TotalAbsPow_Right Temporal__variance,TotalAbsPow_Right Temporal__root_mean_square,TotalAbsPow_Right Temporal__maximum,TotalAbsPow_Right Temporal__absolute_maximum,TotalAbsPow_Right Temporal__minimum
0,178.415884,0.393769,0.392123,455.0,0.085949,0.007387,0.401432,0.699042,0.699042,0.155450,...,9228.968974,18.621076,20.283448,455.0,7.417813,55.023954,21.597274,86.339395,86.339395,9.804144
1,149.004959,0.405619,0.402716,370.0,0.096114,0.009238,0.414027,0.676764,0.676764,0.125978,...,6116.198975,15.369586,16.530267,370.0,5.423472,29.414050,17.397235,47.523821,47.523821,8.094355
2,447.293735,0.595075,0.577153,775.0,0.117833,0.013885,0.589059,0.812816,0.812816,0.119301,...,9841.634697,11.010547,12.698883,775.0,6.356038,40.399221,14.200735,69.739713,69.739713,4.593473
3,238.784690,0.530446,0.530633,450.0,0.071401,0.005098,0.535415,0.791161,0.791161,0.334278,...,12061.262410,22.259106,26.802805,450.0,14.867840,221.052672,30.650335,138.822564,138.822564,10.076186
4,159.357949,0.525363,0.531193,300.0,0.087329,0.007626,0.538324,0.778927,0.778927,0.342696,...,12325.428963,32.864062,41.084763,300.0,25.975822,674.743303,48.607624,181.248867,181.248867,11.984015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,71.248413,0.623983,0.619551,115.0,0.086390,0.007463,0.625545,0.828271,0.828271,0.369688,...,2615.649540,21.104144,22.744779,115.0,8.226251,67.671205,24.186694,55.614061,55.614061,10.430402
65,139.460655,0.433831,0.435815,320.0,0.137168,0.018815,0.456891,0.806495,0.806495,0.120926,...,4878.818348,13.639040,15.246307,320.0,5.883444,34.614914,16.342117,52.595602,52.595602,6.721379
66,111.876932,0.447400,0.447508,250.0,0.112682,0.012697,0.461476,0.726185,0.726185,0.143943,...,4066.194777,14.336679,16.264779,250.0,8.146573,66.366656,18.190923,71.626224,71.626224,7.924013
67,170.184708,0.495245,0.493289,345.0,0.093314,0.008707,0.502037,0.745341,0.745341,0.145035,...,6727.343118,17.359128,19.499545,345.0,8.830897,77.984741,21.406004,81.915691,81.915691,8.720910


In [3]:
#Generate a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())
for region in regions:
    region_features = [col for col in extracted_ts_fresh_df.dropna(axis = 1).columns if region + '_' in col]
    if len(region_features) > 0 : 
        regional_features_dict[region] = region_features

## Expt 1

In [12]:
#Run Expt 1 experiments 

t1 = time.time()

expt_1_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type] 
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'SVC' : SVC(), 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_1_results[data_type] = results_df

t2 = time.time()

t2-t1

102.99936652183533

In [13]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_1_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'SVC' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.616667,0.6,0.666667,0.533333,0.45,0.583333,0.583333,0.516667,0.55,0.616667,0.616667,0.566667,0.666667


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.6,0.6,0.583333,0.6,0.533333,0.55,0.6,0.566667,0.533333,0.633333,0.633333,0.7,0.65


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.55,0.783333,0.7,0.666667,0.55,0.616667,0.6,0.6,0.533333,0.616667,0.516667,0.583333,0.566667


N1


...

,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.465278,0.461111,0.522222,0.434722,0.436111,0.469444,0.411111,0.594444,0.425,0.508333,0.541667,0.45,0.520833


REM


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.5375,0.4375,0.5125,0.45,0.5625,0.5625,0.525,0.5,0.375,0.5,0.525,0.4625,0.4625


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.65,0.65,0.65,0.6375,0.65,0.65,0.65,0.625,0.625,0.65,0.65,0.6375,0.6375


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.575,0.5,0.5125,0.3625,0.5625,0.575,0.5375,0.45,0.35,0.475,0.55,0.4875,0.55


## Expt 2 

In [14]:
#Run Expt 2 experiments 

t1 = time.time()

expt_2_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'SVC' : SVC(), 'Ada_B' : AdaBoostClassifier(random_state = 5)}, return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_2_results[data_type] = results_df

t2 = time.time()

t2-t1

105.87876677513123

In [20]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_2_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'SVC' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.7,0.614286,0.576786,0.6375,0.391071,0.473214,0.407143,0.378571,0.351786,0.469643,0.578571,0.501786,0.528571


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.601786,0.475,0.5625,0.45,0.625,0.623214,0.553571,0.460714,0.514286,0.5,0.528571,0.508929,0.514286


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.6625,0.607143,0.591071,0.603571,0.503571,0.407143,0.433929,0.464286,0.444643,0.489286,0.476786,0.467857,0.5


N1


...

,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.448889,0.452222,0.545556,0.478889,0.421111,0.478889,0.364444,0.463333,0.381111,0.357778,0.376667,0.457778,0.54


REM


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.488889,0.447778,0.394444,0.468889,0.467778,0.465556,0.48,0.456667,0.381111,0.406667,0.403333,0.393333,0.381111


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.49,0.49,0.52,0.458889,0.541111,0.49,0.47,0.478889,0.488889,0.478889,0.478889,0.478889,0.51


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.511111,0.468889,0.503333,0.417778,0.491111,0.553333,0.563333,0.54,0.424444,0.454444,0.445556,0.413333,0.502222


## Expt 3

In [16]:
#Run Expt 3 experiments 

t1 = time.time()

expt_3_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3)

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'SVC' : SVC(), 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_3_results[data_type] = results_df

t2 = time.time()

t2-t1

1800.8339302539825

In [21]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_3_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'SVC' in col]].mean(axis = 0)).T)
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.493939,0.348485,0.314394,0.355303,0.250758,0.429545,0.354545,0.32197,0.366667,0.385606,...,0.287121,0.311364,0.303788,0.250758,0.27803,0.287121,0.313636,0.251515,0.303788,0.311364


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.493939,0.355303,0.354545,0.385606,0.268182,0.273485,0.25,0.250758,0.265152,0.285606,0.311364,0.27803,0.251515


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.348485,0.250758,0.32197,0.250758,0.287121,0.295455,0.332576,0.27803,0.35,0.294697,0.303788,0.287121,0.303788


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.314394,0.429545,0.366667,0.384848,0.284091,0.357576,0.284091,0.267424,0.249242,0.287121,0.250758,0.313636,0.311364


...

REM


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.428022,0.275275,0.368132,0.436264,0.231319,0.346703,0.442857,0.252747,0.427473,0.434615,...,0.32033,0.357143,0.238462,0.347802,0.354396,0.281319,0.348901,0.305495,0.245604,0.379121


,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.428022,0.436264,0.442857,0.434615,0.413187,0.377473,0.427473,0.36978,0.356044,0.340659,0.357143,0.354396,0.305495


,Prefrontal_SVC_acc,Frontal_SVC_acc,Left Frontal_SVC_acc,Right Frontal_SVC_acc,Central_SVC_acc,Left Central_SVC_acc,Right Central_SVC_acc,Left Temporal_SVC_acc,Right Temporal_SVC_acc,Parietal_SVC_acc,Left Parietal_SVC_acc,Right Parietal_SVC_acc,Occipital_SVC_acc
0,0.275275,0.231319,0.252747,0.224176,0.297253,0.341209,0.31978,0.304945,0.282967,0.252747,0.238462,0.281319,0.245604


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.368132,0.346703,0.427473,0.421429,0.391209,0.326374,0.369231,0.353846,0.369231,0.32033,0.347802,0.348901,0.379121


## Expt 4

In [18]:
t1 = time.time()

expt_4_results = {}

for data_type in data_types:
    X_y_groups = loaded_data[data_type]
    
    X = X_y_groups['X']
    y = X_y_groups['y'] 
    groups = X_y_groups['groups'] 
    
    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4)

    results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'SVC' : SVC(), 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
    
    expt_4_results[data_type] = results_df

t2 = time.time()

t2-t1

881.6521239280701

In [19]:
#Show the results
for data_type in data_types:
    print(data_type)
    results_df = expt_4_results[data_type]
    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    #print the results 
    # print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0))
    display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0)).T)

Wake


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.367424,0.241667,0.243182,0.372727,0.249242,0.355303,0.364394,0.249242,0.365152,0.437121,...,0.260606,0.300758,0.276515,0.268182,0.275,0.277273,0.232576,0.287121,0.250758,0.29697


N1


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.401905,0.508095,0.345238,0.409524,0.501905,0.309524,0.409524,0.55,0.345238,0.416667,...,0.248571,0.333333,0.445238,0.318095,0.310952,0.450952,0.338095,0.295238,0.401905,0.310952


N2


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.269048,0.24,0.330476,0.274286,0.218571,0.324286,0.169048,0.24,0.297143,0.288571,...,0.353333,0.319524,0.29,0.374286,0.275238,0.311429,0.312381,0.324286,0.282857,0.338095


N3


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.192381,0.229048,0.234762,0.213333,0.228571,0.25619,0.184762,0.208571,0.29,0.267143,...,0.23619,0.180952,0.222857,0.235238,0.20619,0.21619,0.291429,0.187143,0.223333,0.270952


REM


,Prefrontal_RF_acc,Prefrontal_SVC_acc,Prefrontal_Ada_B_acc,Frontal_RF_acc,Frontal_SVC_acc,Frontal_Ada_B_acc,Left Frontal_RF_acc,Left Frontal_SVC_acc,Left Frontal_Ada_B_acc,Right Frontal_RF_acc,...,Parietal_Ada_B_acc,Left Parietal_RF_acc,Left Parietal_SVC_acc,Left Parietal_Ada_B_acc,Right Parietal_RF_acc,Right Parietal_SVC_acc,Right Parietal_Ada_B_acc,Occipital_RF_acc,Occipital_SVC_acc,Occipital_Ada_B_acc
0,0.392857,0.310989,0.368132,0.413187,0.274176,0.348901,0.420879,0.274176,0.313187,0.362088,...,0.341209,0.42033,0.281319,0.383516,0.354945,0.338462,0.325824,0.28956,0.295604,0.347802
